In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
model= ChatOpenAI()

In [2]:
class qa(TypedDict):
    topic:str
    outline:str
    content:str

In [4]:
graph= StateGraph(qa)

In [ ]:
def create_outline(state: qa) -> qa:
    #fetch topic
    topic = state['title']
    
    #make prompt
    prompt = f'make outline for this topic - {topic}'
    outline = model.invoke(prompt).content
    
    #create outline
    state['outline'] = outline

In [ ]:
def create_blog(state: qa) -> qa:
    #fetch outline
    outline = state['outline']
    
    #make prompt
    prompt = f'write a blog post based on this outline - {outline}'
    content = model.invoke(prompt).content
    
    #create blog
    state['content'] = content

In [ ]:
graph.add_node('create outline', create_outline)
graph.add_node('create blog', create_blog)
graph.add_edge(START, 'create outline')
graph.add_edge('create outline', 'create blog')
graph.add_edge('create blog', END)


In [ ]:
workflow = graph.compile()
initial_state: qa = {'topic': 'The future of AI in healthcare', 'outline': '', 'content': ''}
final_state = workflow(initial_state)